# 02 — Copula Families (Unified Analysis Notebook)
This notebook programmatically analyzes and **generates figures** for the 10 chapters on copulas
you provided. It saves every image exactly to the **paths referenced in the chapters** 



In [18]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

BASE = Path(".").resolve().parent  # adjust when running inside notebooks/
DOCS = BASE / "docs" / "assets" / "figures"
(DOCS / "02_families").mkdir(parents=True, exist_ok=True)

def savefig(path, **kwargs):
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    plt.tight_layout()
    plt.savefig(path, **kwargs)
    plt.close()

def scatter_uv(U, title, savepath, s=0.2, alpha=0.3, lim=(0,1)):
    plt.figure(figsize=(5,4))
    plt.scatter(U[:,0], U[:,1], s=s, alpha=alpha, edgecolors="none")
    plt.xlim(lim); plt.ylim(lim)
    plt.xlabel("u"); plt.ylabel("v"); plt.title(title)
    savefig(savepath, format=Path(savepath).suffix.replace(".","")[0:] or "png")

def contour_C(func, params, title, savepath, N=300):
    u = np.linspace(1e-4, 1-1e-4, N)
    U,V = np.meshgrid(u,u, indexing="xy")
    C = func(U,V,**params)
    plt.figure(figsize=(5,4))
    cs = plt.contour(U, V, C, levels=np.linspace(0.1,0.9,9))
    plt.clabel(cs, inline=1, fontsize=8, fmt="%.1f")
    plt.xlabel("u"); plt.ylabel("v"); plt.title(title)
    savefig(savepath, format=Path(savepath).suffix.replace(".","")[0:] or "png")

def corr_to_cov(rho): return np.array([[1.0, rho],[rho,1.0]])

def gaussian_copula_sample(n=20000, rho=0.6, seed=0):
    rng = np.random.default_rng(seed)
    z = rng.multivariate_normal([0,0], corr_to_cov(rho), size=n)
    try:
        from scipy.special import ndtr
        u = ndtr(z)
    except Exception:
        u = 0.5*(1.0 + np.erf(z/np.sqrt(2.0)))
    return u

def student_t_copula_sample(n=20000, rho=0.6, nu=4, seed=1):
    rng = np.random.default_rng(seed)
    z = rng.multivariate_normal([0,0], corr_to_cov(rho), size=n)
    s = rng.chisquare(df=nu, size=n)/nu
    y = z/np.sqrt(s)[:,None]
    try:
        from scipy.stats import t as tdist
        u = tdist.cdf(y, df=nu)
    except Exception:
        u = 0.5*(1.0 + np.erf(y/np.sqrt(2.0)))
    return u

def clayton_copula_sample(n=20000, theta=2.0, seed=2):
    rng = np.random.default_rng(seed)
    W = rng.gamma(shape=1.0/theta, scale=1.0, size=n)
    E1 = rng.exponential(scale=1.0, size=n); E2 = rng.exponential(scale=1.0, size=n)
    U1 = (1.0 + E1/W)**(-1.0/theta); U2 = (1.0 + E2/W)**(-1.0/theta)
    return np.column_stack([U1,U2])

def gumbel_C(u,v,theta): return np.exp(-(((-np.log(u))**theta + (-np.log(v))**theta)**(1.0/theta)))
def frank_C(u,v,theta):
    if abs(theta)<1e-8: return u*v
    num = (np.exp(-theta*u)-1.0)*(np.exp(-theta*v)-1.0); den = np.exp(-theta)-1.0
    return -1.0/theta * np.log(1.0 + num/den)
def joe_C(u,v,theta): return 1.0 - ((1.0 - (1.0-u)**theta)*(1.0 - (1.0-v)**theta))**(1.0/theta)
def amh_C(u,v,theta): return (u*v)/ (1.0 - theta*(1.0-u)*(1.0-v))
def plackett_C(u,v,theta):
    if abs(theta-1.0)<1e-12: return u*v
    term = 1 + (theta - 1)*(u+v)
    s = np.sqrt(term**2 - 4*u*v*theta*(theta-1))
    return (term - s) / (2*(theta-1))
def fgm_C(u,v,theta): return u*v*(1.0 + theta*(1.0-u)*(1.0-v))

def tawn_C(u,v,theta,alpha,beta):
    t = np.log(v)/np.log(u*v)
    A = (1-alpha)*(1-t) + (1-beta)*t + ((alpha*(1-t))**theta + (beta*t)**theta)**(1.0/theta)
    return np.exp(np.log(u*v)*A)


## Gaussian

In [19]:

# Contours
U,V,dens = None, None, None
# Use density-like contours via latent ellipse
u = np.linspace(1e-4, 1-1e-4, 350)
U,V = np.meshgrid(u,u, indexing="xy")
rho=0.6
try:
    from scipy.stats import norm
    z1 = norm.ppf(U); z2 = norm.ppf(V)
except Exception:
    z1 = np.sqrt(2)*np.erfinv(2*U-1); z2 = np.sqrt(2)*np.erfinv(2*V-1)
inv = np.linalg.inv(corr_to_cov(rho))
quad = inv[0,0]*z1*z1 + inv[1,1]*z2*z2 + 2*inv[0,1]*z1*z2
dens = np.exp(-0.5*quad)/ (2*np.pi*np.sqrt(1-rho**2))

plt.figure(figsize=(5,4))
cs = plt.contour(U,V,dens, levels=np.linspace(dens.min()+1e-6, np.percentile(dens, 98), 8))
plt.clabel(cs, inline=1, fontsize=8, fmt="%.2f")
plt.xlabel("u"); plt.ylabel("v"); plt.title("Gaussian copula (ρ=0.6) — pseudo-density contours")
savefig(DOCS/"02_families"/"gaussian_copula_contours.svg", format="svg")

# Samples
U = gaussian_copula_sample(n=30000, rho=0.6, seed=1)
scatter_uv(U, "Gaussian copula samples (ρ=0.6)", DOCS/"02_families"/"gaussian_copula_samples.svg", s=0.2, alpha=0.3)


## Student‑t

In [20]:

# Contours via histogram of samples
U = student_t_copula_sample(n=60000, rho=0.6, nu=4, seed=2)
H, xe, ye = np.histogram2d(U[:,0], U[:,1], bins=220, range=[[0,1],[0,1]], density=True)
Xc, Yc = np.meshgrid(0.5*(xe[:-1]+xe[1:]), 0.5*(ye[:-1]+ye[1:]), indexing="xy")
plt.figure(figsize=(5,4))
cs = plt.contour(Xc, Yc, H.T, levels=np.linspace(H.min(), np.percentile(H, 97), 8))
plt.clabel(cs, inline=1, fontsize=8, fmt="%.2f")
plt.xlabel("u"); plt.ylabel("v"); plt.title("Student-t copula (ρ=0.6, ν=4) — density contours")
savefig(DOCS/"02_families"/"student_t_copula_contours.svg", format="svg")

# Samples
U = student_t_copula_sample(n=40000, rho=0.6, nu=4, seed=3)
scatter_uv(U, "Student-t copula samples (ρ=0.6, ν=4)", DOCS/"02_families"/"student_t_copula_samples.svg", s=0.2, alpha=0.3)

# Tail dependence curves (saved in 01_foundations)
rho_vals = np.linspace(-0.95, 0.95, 161); nus = [3,4,6,10]
plt.figure(figsize=(6,4))
for nu in nus:
    try:
        from scipy.stats import t as tdist
        inner = -np.sqrt(((nu+1.0)/(1.0+rho_vals))*(1.0-rho_vals))
        lam = 2.0*tdist.cdf(inner, df=nu+1.0)
    except Exception:
        inner = -np.sqrt(((nu+1.0)/(1.0+rho_vals))*(1.0-rho_vals))
        lam = 2.0*0.5*(1.0 + np.erf(inner/np.sqrt(2.0)))
    plt.plot(rho_vals, lam, label=f"nu={nu}")
plt.xlabel(r"$\rho$"); plt.ylabel(r"$\lambda_U=\lambda_L$"); plt.title("Tail dependence of t-copula")
plt.legend(); savefig(DOCS/"01_foundations"/"t_lambda_dependence_all.svg", format="svg")


## Clayton

In [21]:

# Contours
contour_C(lambda u,v,theta: ((u**(-theta)+v**(-theta)-1.0)**(-1.0/theta)),
          {"theta":2.0}, "Clayton copula contours (θ=2)", DOCS/"02_families"/"clayton_copula_contours.svg")
# Samples
U = clayton_copula_sample(n=40000, theta=2.0, seed=11)
scatter_uv(U, "Clayton copula samples (θ=2)", DOCS/"02_families"/"clayton_copula_samples.svg", s=0.2, alpha=0.3)


## Gumbel

In [30]:
# Contornos
contour_C(
    gumbel_C,
    {"theta": 2.0},
    "Gumbel copula contours (θ=2)",
    DOCS / "02_families" / "gumbel_copula_contours.svg",
)

# Curva de cola superior λ_U vs θ
th = np.linspace(1.0, 6.0, 200)
lam = 2.0 - 2.0 ** (1.0 / th)

plt.figure(figsize=(5, 4))
plt.plot(th, lam, label=r"$\lambda_U$")
plt.xlabel(r"$\theta$")
plt.ylabel(r"$\lambda_U$")
plt.title(r"Gumbel upper-tail dependence vs $\theta$")
plt.grid(True, linewidth=0.5, alpha=0.5)
plt.legend()

# Si usas tu helper savefig que llama a tight_layout internamente:
savefig(DOCS / "02_families" / "gumbel_lambda_vs_theta.svg", format="svg")


## Joe

In [ ]:

contour_C(joe_C, {"theta":2.0}, "Joe copula contours (θ=2)", DOCS/"02_families"/"joe_copula_contours.svg")
# Same lambda curve as Gumbel
th = np.linspace(1.0, 6.0, 200); lam = 2.0 - 2.0**(1.0/th)
plt.figure(figsize=(5,4)); plt.plot(th, lam)
plt.xlabel(r"$\theta$"); plt.ylabel(r"$\lambda_U$"); plt.title("Joe upper-tail dependence vs θ")
savefig(DOCS/"02_families"/"joe_lambda_vs_theta.svg", format="svg")
def joe_C(u, v, theta):
    # Joe copula (forma correcta):
    # C(u,v) = 1 - ( (1-u)^θ + (1-v)^θ - (1-u)^θ (1-v)^θ )^{1/θ}
    u = np.clip(u, 1e-12, 1 - 1e-12)
    v = np.clip(v, 1e-12, 1 - 1e-12)
    A = (1.0 - u)**theta
    B = (1.0 - v)**theta
    S = A + B - A * B
    return 1.0 - np.power(np.clip(S, 1e-16, None), 1.0 / theta)

def joe_dC_du(u, v, theta):
    # dC/du para inversión condicional (monótona creciente en v)
    u = np.clip(u, 1e-12, 1 - 1e-12)
    v = np.clip(v, 1e-12, 1 - 1e-12)
    A = (1.0 - u)**theta
    B = (1.0 - v)**theta
    S = A + B - A * B
    dA_du = -theta * (1.0 - u)**(theta - 1.0)           # < 0
    dS_du = dA_du * (1.0 - B)                            # ≤ 0
    # dC/du = - (1/θ) * S^{1/θ - 1} * dS/du  => ≥ 0
    return - (1.0 / theta) * np.power(np.clip(S, 1e-16, None), 1.0 / theta - 1.0) * dS_du

def joe_conditional_sample_v(u, z, theta, tol=1e-6, maxit=64):
    # Bisección en v ∈ (0,1) resolviendo ∂C/∂u(u,v)=z
    lo, hi = 1e-12, 1.0 - 1e-12
    for _ in range(maxit):
        mid = 0.5 * (lo + hi)
        f_mid = joe_dC_du(u, mid, theta)
        if f_mid < z:
            lo = mid
        else:
            hi = mid
        if hi - lo < tol:
            break
    return 0.5 * (lo + hi)

# --- Muestras ---
theta = 2.0
rng = np.random.default_rng(2025)
n = 18000
u = rng.random(n)
z = rng.random(n)
v = np.empty(n)
for i in range(n):
    v[i] = joe_conditional_sample_v(u[i], z[i], theta)

plt.figure(figsize=(5, 4))
plt.scatter(u, v, s=0.2, alpha=0.3, edgecolors="none")
plt.xlim(0, 1); plt.ylim(0, 1)
plt.xlabel("u"); plt.ylabel("v")
plt.title(f"Joe copula samples (θ={theta:g})")
savefig(DOCS / "02_families" / "joe_copula_samples.svg", format="svg")

# --- λ_U(θ) — curva de dependencia de cola superior ---
th = np.linspace(1.0, 6.0, 200)
lamU = 2.0 - np.power(2.0, 1.0 / th)   # Joe (igual que Gumbel), λ_L = 0

plt.figure(figsize=(5, 4))
plt.plot(th, lamU, label=r"$\lambda_U$")
plt.xlabel(r"$\theta$")
plt.ylabel(r"$\lambda_U$")
plt.title(r"Joe upper-tail dependence vs $\theta$")
plt.grid(True, linewidth=0.5, alpha=0.5)
plt.legend()
savefig(DOCS / "02_families" / "joe_lambda_vs_theta.svg", format="svg")


## Frank

In [33]:
## Frank

def frank_C(u, v, theta):
    if abs(theta) < 1e-8:
        return u * v
    num = (np.exp(-theta * u) - 1.0) * (np.exp(-theta * v) - 1.0)
    den = np.exp(-theta) - 1.0
    return -1.0 / theta * np.log1p(num / den)  # log1p mejora estabilidad numérica


# === Contornos ===
contour_C(
    frank_C,
    {"theta": 6.0},
    "Frank copula contours (θ=6)",
    DOCS / "02_families" / "frank_copula_contours.svg",
)


# === Muestras (inversión condicional) ===
def frank_dC_du(u, v, theta):
    A = np.exp(-theta * u)
    B = np.exp(-theta * v)
    D = np.exp(-theta) - 1.0
    Q = 1.0 + ((A - 1.0) * (B - 1.0)) / D
    # dC/du = (A * (B - 1)) / (D * Q)
    return (A * (B - 1.0)) / (D * Q)

def frank_conditional_sample_v(u, z, theta, tol=1e-7, maxit=64):
    lo, hi = 1e-12, 1 - 1e-12
    for _ in range(maxit):
        mid = 0.5 * (lo + hi)
        f_mid = frank_dC_du(u, mid, theta)
        if f_mid < z:
            lo = mid
        else:
            hi = mid
        if hi - lo < tol:
            break
    return 0.5 * (lo + hi)

theta = 6.0
rng = np.random.default_rng(123)
n = 18000
u = rng.random(n)
z = rng.random(n)
v = np.empty(n)
for i in range(n):
    v[i] = frank_conditional_sample_v(u[i], z[i], theta)

plt.figure(figsize=(5, 4))
plt.scatter(u, v, s=0.2, alpha=0.3, edgecolors="none")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel("u")
plt.ylabel("v")
plt.title(f"Frank copula samples (θ={theta:g})")
savefig(DOCS / "02_families" / "frank_copula_samples.svg", format="svg")


# === τ de Kendall vs θ ===
def frank_tau(theta):
    """Kendall's tau closed form via Debye function."""
    from scipy.integrate import quad
    # Debye D1(t) = (1/t) * ∫_0^t (x / (exp(x) - 1)) dx
    def D1(t):
        if t == 0:
            return 1.0
        val, _ = quad(lambda x: x / (np.exp(x) - 1.0), 0, t)
        return val / t
    return 1.0 - 4.0 * (1.0 / theta - D1(theta) / theta)

thetas = np.linspace(0.1, 10, 200)
taus = [frank_tau(t) for t in thetas]

plt.figure(figsize=(5, 4))
plt.plot(thetas, taus, label=r"$\tau(\theta)$")
plt.xlabel(r"$\theta$")
plt.ylabel(r"$\tau$")
plt.title("Kendall's tau vs θ — Frank copula")
plt.grid(True, linewidth=0.5, alpha=0.5)
plt.legend()
savefig(DOCS / "02_families" / "frank_tau_vs_theta.svg", format="svg")



## AMH

In [35]:

contour_C(amh_C, {"theta":0.8}, "AMH copula contours (θ=0.8)", DOCS/"02_families"/"amh_copula_contours.svg")

## AMH — samples + Kendall's tau vs θ

def amh_C(u, v, theta):
    # θ ∈ [-1, 1)
    denom = 1.0 - theta * (1.0 - u) * (1.0 - v)
    return (u * v) / np.clip(denom, 1e-12, None)

# dC/du para inversión condicional (monótono en v)
def amh_dC_du(u, v, theta):
    denom = 1.0 - theta * (1.0 - u) * (1.0 - v)
    denom = np.clip(denom, 1e-12, None)
    # dC/du = v/den + u v * θ(1-v) / den^2
    term1 = v / denom
    term2 = u * v * theta * (1.0 - v) / (denom**2)
    return term1 + term2

def amh_conditional_sample_v(u, z, theta, tol=1e-6, maxit=64):
    # Resuelve dC/du(u, v, θ) = z en v∈(0,1) por bisección
    lo, hi = 1e-12, 1.0 - 1e-12
    for _ in range(maxit):
        mid = 0.5 * (lo + hi)
        f_mid = amh_dC_du(u, mid, theta)
        if f_mid < z:
            lo = mid
        else:
            hi = mid
        if hi - lo < tol:
            break
    return 0.5 * (lo + hi)

# --- Muestras (θ=0.8) ---
theta = 0.8
rng = np.random.default_rng(2025)
n = 18000
u = rng.random(n)
z = rng.random(n)
v = np.empty(n)
for i in range(n):
    v[i] = amh_conditional_sample_v(u[i], z[i], theta)

plt.figure(figsize=(5, 4))
plt.scatter(u, v, s=0.2, alpha=0.3, edgecolors="none")
plt.xlim(0, 1); plt.ylim(0, 1)
plt.xlabel("u"); plt.ylabel("v")
plt.title(f"AMH copula samples (θ={theta:g})")
savefig(DOCS / "02_families" / "amh_copula_samples.svg", format="svg")

# --- Kendall's τ(θ) (estimación Monte Carlo robusta) ---
def amh_tau_mc(theta, n=6000, seed=0):
    """Estimador de τ=4 E[C(U,V)]-1 para AMH por Monte Carlo."""
    rng = np.random.default_rng(seed)
    U = rng.random(n)
    Z = rng.random(n)
    V = np.empty(n)
    for i in range(n):
        V[i] = amh_conditional_sample_v(U[i], Z[i], theta)
    Cuv = amh_C(U, V, theta)
    return float(4.0 * np.mean(Cuv) - 1.0)

thetas = np.linspace(-0.9, 0.95, 25)  # AMH válido en [-1, 1)
taus = [amh_tau_mc(t, n=6000, seed=123 + i) for i, t in enumerate(thetas)]

plt.figure(figsize=(5, 4))
plt.plot(thetas, taus, marker=".", lw=1.25, label=r"$\tau(\theta)$")
plt.axhline(0.0, ls="--", lw=0.8)
plt.xlabel(r"$\theta$")
plt.ylabel(r"$\tau$")
plt.title("Kendall's tau vs θ — AMH copula (MC)")
plt.grid(True, linewidth=0.5, alpha=0.5)
plt.legend()
savefig(DOCS / "02_families" / "amh_tau_vs_theta.svg", format="svg")


## Advanced BB Families

In [26]:

def bb1_C(u,v,theta,delta):
    return (1.0 + ((u**(-theta)-1.0)**delta + (v**(-theta)-1.0)**delta)**(1.0/delta))**(-1.0/theta)
def bb7_C(u,v,theta,delta):
    term = (1 - (1-u)**theta)**(-delta) + (1 - (1-v)**theta)**(-delta) - 1.0
    return 1.0 - (1.0 - term**(-1.0/delta))**(1.0/theta)

# Combined comparison figure
u = np.linspace(1e-4, 1-1e-4, 250)
U,V = np.meshgrid(u,u, indexing="xy")
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
C1 = bb1_C(U,V,theta=2.0, delta=1.5)
cs = plt.contour(U,V,C1, levels=np.linspace(0.1,0.9,9))
plt.clabel(cs, inline=1, fontsize=7); plt.title("BB1 (θ=2, δ=1.5)"); plt.xlabel("u"); plt.ylabel("v")
plt.subplot(1,2,2)
C2 = bb7_C(U,V,theta=2.0, delta=1.2)
cs2 = plt.contour(U,V,C2, levels=np.linspace(0.1,0.9,9))
plt.clabel(cs2, inline=1, fontsize=7); plt.title("BB7 (θ=2, δ=1.2)"); plt.xlabel("u"); plt.ylabel("v")
savefig(DOCS/"02_families"/"bb_family_contours.svg", format="svg")


In [43]:
## BB families — tail dependence comparison (clean)

def lambda_L_BB1(theta, delta):
    theta = np.asarray(theta, float)
    return 2.0 ** (-1.0 / theta)          # depende de θ

def lambda_U_BB1(theta, delta):
    delta = np.asarray(delta, float)
    return 2.0 - 2.0 ** (1.0 / delta)     # depende de δ

def lambda_L_BB7(theta, delta):
    delta = np.asarray(delta, float)
    return 2.0 ** (-1.0 / delta)          # depende de δ

def lambda_U_BB7(theta, delta):
    theta = np.asarray(theta, float)
    return 2.0 - 2.0 ** (1.0 / theta)     # depende de θ

thetas  = np.linspace(1.05, 6.0, 200)
deltas  = np.linspace(1.05, 6.0, 200)
delta_f = 1.5
theta_f = 2.0

plt.figure(figsize=(10,4))

# Panel izquierdo: barrer θ  -> mostrar λ que dependen de θ
plt.subplot(1,2,1)
plt.plot(thetas, lambda_L_BB1(thetas, delta_f), label=r"BB1 $\lambda_L(\theta;\,\delta=1.5)$")
plt.plot(thetas, lambda_U_BB7(thetas, delta_f), label=r"BB7 $\lambda_U(\theta;\,\delta=1.5)$")
plt.xlabel(r"$\theta$"); plt.ylabel(r"$\lambda$"); plt.ylim(0,1)
plt.title(r"Tail dependence varying $\theta$ (fix $\delta=1.5$)")
plt.grid(True, linewidth=0.5, alpha=0.5); plt.legend(fontsize=8)

# Panel derecho: barrer δ  -> mostrar λ que dependen de δ
plt.subplot(1,2,2)
plt.plot(deltas, lambda_U_BB1(theta_f, deltas), label=r"BB1 $\lambda_U(\delta;\,\theta=2)$")
plt.plot(deltas, lambda_L_BB7(theta_f, deltas), label=r"BB7 $\lambda_L(\delta;\,\theta=2)$")
plt.xlabel(r"$\delta$"); plt.ylabel(r"$\lambda$"); plt.ylim(0,1)
plt.title(r"Tail dependence varying $\delta$ (fix $\theta=2$)")
plt.grid(True, linewidth=0.5, alpha=0.5); plt.legend(fontsize=8)

savefig(DOCS / "02_families" / "bb_tail_dependence_comparison.svg", format="svg")


In [37]:
## BB families — copula samples (BB1 & BB7)

# ===== BB1 dC/du y muestreo =====
def dCdu_BB1(u, v, theta, delta):
    u = np.clip(u, 1e-12, 1-1e-12)
    v = np.clip(v, 1e-12, 1-1e-12)
    A = u**(-theta) - 1.0
    B = v**(-theta) - 1.0
    S_base = A**delta + B**delta
    S = np.power(np.clip(S_base, 1e-16, None), 1.0/delta)
    C = np.power(1.0 + S, -1.0/theta)
    return C/(1.0+S) * np.power(S_base, 1.0/delta - 1.0) * np.power(A, delta-1.0) * u**(-theta-1.0)

def sample_BB1(n, theta=2.0, delta=1.5, seed=10, tol=1e-6, maxit=64):
    rng = np.random.default_rng(seed)
    u = rng.random(n)
    z = rng.random(n)
    v = np.empty(n)
    for i in range(n):
        lo, hi = 1e-12, 1.0 - 1e-12
        for _ in range(maxit):
            mid = 0.5 * (lo + hi)
            f_mid = dCdu_BB1(u[i], mid, theta, delta)
            if f_mid < z[i]:
                lo = mid
            else:
                hi = mid
            if hi - lo < tol:
                break
        v[i] = 0.5 * (lo + hi)
    return np.column_stack([u, v])

# ===== BB7 dC/du y muestreo =====
def dCdu_BB7(u, v, theta, delta):
    u = np.clip(u, 1e-12, 1-1e-12)
    v = np.clip(v, 1e-12, 1-1e-12)
    S_u = 1.0 - (1.0 - u)**theta
    S_v = 1.0 - (1.0 - v)**theta
    A = np.power(np.clip(S_u, 1e-16, None), -delta)
    B = np.power(np.clip(S_v, 1e-16, None), -delta)
    T = A + B - 1.0
    H = 1.0 - np.power(np.clip(T, 1e-16, None), -1.0/delta)
    return np.power(np.clip(H, 1e-16, None), 1.0/theta - 1.0) * \
           np.power(np.clip(T, 1e-16, None), -1.0/delta - 1.0) * \
           np.power(1.0 - u, theta - 1.0) * \
           np.power(np.clip(S_u, 1e-16, None), -delta - 1.0)

def sample_BB7(n, theta=2.0, delta=1.2, seed=11, tol=1e-6, maxit=64):
    rng = np.random.default_rng(seed)
    u = rng.random(n)
    z = rng.random(n)
    v = np.empty(n)
    for i in range(n):
        lo, hi = 1e-12, 1.0 - 1e-12
        for _ in range(maxit):
            mid = 0.5 * (lo + hi)
            f_mid = dCdu_BB7(u[i], mid, theta, delta)
            if f_mid < z[i]:
                lo = mid
            else:
                hi = mid
            if hi - lo < tol:
                break
        v[i] = 0.5 * (lo + hi)
    return np.column_stack([u, v])

# --- Generar y guardar figura lado a lado ---
U1 = sample_BB1(n=18000, theta=2.0, delta=1.5, seed=101)
U7 = sample_BB7(n=18000, theta=2.0, delta=1.2, seed=202)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.scatter(U1[:,0], U1[:,1], s=0.2, alpha=0.3, edgecolors="none")
plt.title("BB1 samples (θ=2, δ=1.5)")
plt.xlim(0,1); plt.ylim(0,1); plt.xlabel("u"); plt.ylabel("v")

plt.subplot(1,2,2)
plt.scatter(U7[:,0], U7[:,1], s=0.2, alpha=0.3, edgecolors="none")
plt.title("BB7 samples (θ=2, δ=1.2)")
plt.xlim(0,1); plt.ylim(0,1); plt.xlabel("u"); plt.ylabel("v")

savefig(DOCS / "02_families" / "bb_copula_samples.svg", format="svg")


## Tawn / Extreme-Value

In [44]:

## Tawn — samples + Pickands + contours

# Pickands function (ya existente)
t = np.linspace(0, 1, 300)
theta = 2.0; alpha = 0.7; beta = 1.0
A = (1 - alpha) * (1 - t) + (1 - beta) * t + ((alpha * (1 - t))**theta + (beta * t)**theta)**(1.0 / theta)

plt.figure(figsize=(5, 4))
plt.plot(t, A)
plt.xlabel("t"); plt.ylabel("A(t)")
plt.title("Tawn Pickands function (θ=2, α=0.7, β=1)")
plt.ylim(0.45, 1.05)
savefig(DOCS / "02_families" / "tawn_pickands_function.svg", format="svg")

# Copula function
def tawn_C(u, v, theta, alpha, beta):
    u = np.clip(u, 1e-8, 1 - 1e-8)
    v = np.clip(v, 1e-8, 1 - 1e-8)
    t = np.log(v) / np.log(u * v)
    A = ((1 - alpha) * (1 - t)
         + (1 - beta) * t
         + ((alpha * (1 - t))**theta + (beta * t)**theta)**(1.0 / theta))
    return np.exp(np.log(u * v) * A)

# Contornos
contour_C(
    tawn_C,
    {"theta": 2.0, "alpha": 0.7, "beta": 1.0},
    "Tawn copula contours (θ=2, α=0.7, β=1)",
    DOCS / "02_families" / "tawn_copula_contours.svg",
)

# === Muestras (EV-sampling approximation) ===
def sample_tawn(n=20000, theta=2.0, alpha=0.7, beta=1.0, seed=42):
    """Approximate sampling for Tawn copula via asymmetric exponential mixture."""
    rng = np.random.default_rng(seed)
    # Base Gumbel-like dependence
    W = rng.weibull(theta, n)
    U = np.exp(- (rng.random(n) ** (1 / (alpha + 1e-6))) / (W + 1e-8))
    V = np.exp(- (rng.random(n) ** (1 / (beta + 1e-6))) / (W + 1e-8))
    # Normalize to (0,1)
    U = (U - U.min()) / (U.max() - U.min())
    V = (V - V.min()) / (V.max() - V.min())
    return np.column_stack([U, V])

# Generar muestras y gráfico
U = sample_tawn(n=18000, theta=2.0, alpha=0.7, beta=1.0, seed=2025)
plt.figure(figsize=(5, 4))
plt.scatter(U[:, 0], U[:, 1], s=0.2, alpha=0.3, edgecolors="none")
plt.xlim(0, 1); plt.ylim(0, 1)
plt.xlabel("u"); plt.ylabel("v")
plt.title("Tawn copula samples (θ=2, α=0.7, β=1)")
savefig(DOCS / "02_families" / "tawn_copula_samples.svg", format="svg")


## Plackett & FGM

In [28]:

contour_C(plackett_C, {"theta":5.0}, "Plackett copula contours (θ=5)", DOCS/"02_families"/"plackett_copula_contours.svg")
contour_C(fgm_C, {"theta":0.8}, "FGM copula contours (θ=0.8)", DOCS/"02_families"/"fgm_copula_contours.svg")

# Illustrative sample figure (side-by-side) to match 'misc_copula_samples.svg'
rng = np.random.default_rng(7); n=20000; U = rng.random((n,2))
P = np.column_stack([U[:,0], 0.6*U[:,0] + 0.4*U[:,1]])
V = np.column_stack([U[:,0], fgm_C(U[:,0], U[:,1], 0.8)])
fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(1,2,1); ax1.scatter(P[:,0], P[:,1], s=0.2, alpha=0.3, edgecolors="none")
ax1.set_title("Plackett-like samples (illustrative)"); ax1.set_xlim(0,1); ax1.set_ylim(0,1)
ax2 = fig.add_subplot(1,2,2); ax2.scatter(V[:,0], V[:,1], s=0.2, alpha=0.3, edgecolors="none")
ax2.set_title("FGM-like samples (illustrative)"); ax2.set_xlim(0,1); ax2.set_ylim(0,1)
for ax in (ax1,ax2):
    ax.set_xlabel("u"); ax.set_ylabel("v")
savefig(DOCS/"02_families"/"misc_copula_samples.svg", format="svg")
